In [1]:
import os
import pickle
import numpy as np

from IPython.display import clear_output

import matplotlib
#import config InlineBackend.figure_format = 'svg'
import tensorflow as tf

# silencing tensorflow warnings
import logging
logging.getLogger('tensorflow').setLevel(logging.FATAL)
from datetime import datetime

tf.__version__ # printint out tensorflow version used
import stable_baselines
from stable_baselines.common.callbacks import BaseCallback
from stable_baselines.results_plotter import load_results, ts2xy
from stable_baselines import PPO2
from stable_baselines.bench import Monitor
from stable_baselines.common.policies import MlpPolicy
from stable_baselines import results_plotter
from stable_baselines.common.evaluation import evaluate_policy
#stable_baselines.__version__ # printing out stable_baselines version used
import gym

/Users/joshnevin/.pyenv/versions/3.7.12/envs/rlenv37/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):
/Users/joshnevin/.pyenv/versions/3.7.12/envs/rlenv37/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [58]:
with open('/Users/joshnevin/RL_FOCSLab/topologies/nsfnet_chen_5-paths_directional.h5', 'rb') as f:
    topology = pickle.load(f)
# node probabilities from https://github.com/xiaoliangchenUCD/DeepRMSA/blob/6708e9a023df1ec05bfdc77804b6829e33cacfe4/Deep_RMSA_A3C.py#L77
node_request_probabilities = np.array([0.01801802, 0.04004004, 0.05305305, 0.01901902, 0.04504505,
       0.02402402, 0.06706707, 0.08908909, 0.13813814, 0.12212212,
       0.07607608, 0.12012012, 0.01901902, 0.16916917])
load = int(1e10)
env_args = dict(topology=topology, seed=1, load = load,
                allow_rejection=False, # the agent cannot proactively reject a request
                mean_service_holding_time=1e6, # value is not set as in the paper to achieve comparable reward values
                episode_length=1000, node_request_probabilities=node_request_probabilities)

In [59]:
env = gym.make('RWAFOCS-v2', **env_args)

In [60]:
model = PPO2.load('/Users/joshnevin/RL_FOCSLab/tmp/RWAFOCS-ppo/2021-12-30_4/best_model')

In [61]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1,deterministic =False)

In [62]:
mean_reward

631.0

In [63]:
std_reward

0.0

In [64]:
print("Whole training process statistics:")
rnd_path_action_probability = np.sum(env.actions_output, axis=1) / np.sum(env.actions_output)
rnd_wavelength_action_probability = np.sum(env.actions_output, axis=0) / np.sum(env.actions_output)
print('Path action probability:', np.sum(env.actions_output, axis=1) / np.sum(env.actions_output))
print('Wavelength action probability:', np.sum(env.actions_output, axis=0) / np.sum(env.actions_output))
num_lps_reused = env.num_lightpaths_reused
print('Load (Erlangs):', load)
print('Service bit rate (Gb/s):', env.service.bit_rate/1e9)
print('Total number of services:', env.services_processed)
print('Total number of accepted services:', env.services_accepted)
print('Blocking probability:', 1 - env.services_accepted/env.services_processed)
print('Number of services on existing lightpaths:', num_lps_reused)
print('Number of services released:', env.num_lightpaths_released)
print('Number of transmitters on each node:', env.num_transmitters)
print('Number of receivers on each node:', env.num_receivers)

Whole training process statistics:
Path action probability: [0.54992076 0.4437401  0.00475436 0.00158479 0.        ]
Wavelength action probability: [0.0110935  0.00158479 0.0110935  0.00792393 0.00950872 0.0110935
 0.00158479 0.00475436 0.00950872 0.01426307 0.00475436 0.00475436
 0.0110935  0.00633914 0.00158479 0.00792393 0.00950872 0.02377179
 0.0110935  0.00633914 0.00633914 0.00950872 0.00633914 0.00792393
 0.00792393 0.01267829 0.00950872 0.01584786 0.00950872 0.00792393
 0.01426307 0.00633914 0.00792393 0.00633914 0.01267829 0.00633914
 0.01426307 0.00792393 0.01584786 0.01901743 0.00633914 0.022187
 0.0110935  0.01901743 0.01267829 0.01426307 0.01426307 0.01267829
 0.00475436 0.00633914 0.00475436 0.00633914 0.0110935  0.01426307
 0.01267829 0.0110935  0.00792393 0.00950872 0.00950872 0.00950872
 0.02377179 0.00633914 0.01584786 0.00792393 0.00475436 0.00475436
 0.00633914 0.00950872 0.03169572 0.022187   0.00633914 0.01267829
 0.00950872 0.00316957 0.00475436 0.00950872 0.0221

In [65]:
# test and visualize
obs = env.reset()
done = False
total_reward = []
while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    total_reward.append(rewards)

In [66]:
sum(total_reward)

652